hybrid_bm25_sbert_chatbot (Hybrid BM25 + SBERT: Untuk hasil maksimal.)
Penjelasan:
>BM25 mengambil 5 kandidat teratas berdasarkan keyword.
>SBERT menghitung similarity semantic antara pertanyaan user dan kandidat.
>Top 3 jawaban paling mirip ditampilkan, urut berdasarkan kemiripan semantic.

Tahapan yang Ada di Kode
-----------
>1 Load Dataset (read file JSON).
>2 Preprocessing ringan: Tokenisasi dan lowercase untuk BM25.
>3 Indexing: Membuat objek BM25 dan menghasilkan embedding SBERT.
>4 Inference: Mengambil pertanyaan user, melakukan pencarian hybrid, dan mengembalikan jawaban.
>5 STT & TTS: Hanya untuk input/output, tidak ada preprocessing data.


Kesimpulan
------
>1 Tidak ada tahapan preprocessing yang kompleks.
>2 Tidak ada pelatihan model (epoch/training).
>3 Semua model yang digunakan (BM25 & SBERT) hanya untuk pencarian dan inference, bukan training ulang.

Contoh Penulisan di Judul atau Subjudul:
---------
>“Pengembangan ClinicBot Hybrid berbasis BM25 dan BERT untuk Optimalisasi Pembelajaran Klinik Non-Fungsional”

>“ClinicBot: Hybrid Context-Aware Conversational Agent untuk Pembelajaran Klinik Manekin berbasis STT-TTS dan IoT”

In [2]:
import json
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer, util
import pyttsx3
import speech_recognition as sr

In [3]:
# ===== LOAD DATASET =====
with open("data/pertanyaan_jawaban.json", "r", encoding="utf-8") as f:
    faq_data = json.load(f)

faq_questions = [item["pertanyaan"] for item in faq_data]
faq_answers = [item["jawaban"] for item in faq_data]

In [4]:
# ===== INISIALISASI BM25 DAN SBERT =====
tokenized_questions = [q.lower().split() for q in faq_questions]
bm25 = BM25Okapi(tokenized_questions)
sbert_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
faq_embeddings = sbert_model.encode(faq_questions)

In [5]:
# ===== INISIALISASI TTS =====
engine = pyttsx3.init()
engine.setProperty('rate', 145)  # kecepatan bicara

def speak(text):
    engine.say(text)
    engine.runAndWait()

# ===== INISIALISASI STT =====
def listen():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Silakan bicara...")
        audio = recognizer.listen(source, timeout=5)
        print("Mendengarkan...")
    try:
        query = recognizer.recognize_google(audio, language="id-ID")
        print(f"Kamu berkata: {query}")
        return query
    except sr.UnknownValueError:
        print("Maaf, suara tidak terbaca.")
        speak("Maaf, suara tidak terbaca.")
        return ""
    except sr.RequestError:
        print("Gagal menghubungi layanan STT.")
        speak("Gagal menghubungi layanan STT.")
        return ""

In [6]:
def hybrid_search(user_question, bm25_top_n=5, sbert_top_k=3):
    # Step 1. BM25 retrieval
    tokenized_query = user_question.lower().split()
    bm25_scores = bm25.get_scores(tokenized_query)
    bm25_top_indices = sorted(range(len(bm25_scores)), key=lambda i: bm25_scores[i], reverse=True)[:bm25_top_n]

    # Step 2. SBERT rerank
    candidate_questions = [faq_questions[i] for i in bm25_top_indices]
    candidate_answers = [faq_answers[i] for i in bm25_top_indices]
    candidate_embeddings = [faq_embeddings[i] for i in bm25_top_indices]
    user_embedding = sbert_model.encode([user_question])[0]
    cos_sims = util.cos_sim(user_embedding, candidate_embeddings)[0].cpu().tolist()

    # Pair and rerank
    pairs = list(zip(bm25_top_indices, candidate_questions, candidate_answers, cos_sims))
    pairs = sorted(pairs, key=lambda x: x[3], reverse=True)  # sort by SBERT cosine similarity

    # Ambil top K hasil
    return pairs[:sbert_top_k]

if __name__ == "__main__":
    print("Chatbot Hybrid BM25 + SBERT (FAQ + STT & TTS)")
    print("----------------------------------------------")
    mode = input("Pilih mode (1 = ketik, 2 = suara): ")
    while True:
        if mode == "2":
            user_question = listen()
            if user_question == "":
                continue
        else:
            user_question = input("\nTulis pertanyaanmu (atau ketik 'selesai' untuk keluar): ")
        if user_question.lower() == "selesai":
            speak("Okey, senang berbicara dengan anda, Sampai jumpa lagi!")
            break
        results = hybrid_search(user_question)
        print("\nTop jawaban paling relevan:")
        for i, (idx, q, a, score) in enumerate(results, 1):
            print(f"\n{i}. Pertanyaan mirip: {q}")
            print(f"   Jawaban       : {a}")
            print(f"   Skor kemiripan: {score:.4f}")
            if i == 1:  # hanya jawaban teratas di-TTS
                speak(a)

Chatbot Hybrid BM25 + SBERT (FAQ + STT & TTS)
----------------------------------------------


Pilih mode (1 = ketik, 2 = suara):  2


Silakan bicara...
Mendengarkan...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Kamu berkata: halo

Top jawaban paling relevan:

1. Pertanyaan mirip: Apa itu BERT?
   Jawaban       : BERT adalah model transformer untuk NLP yang dikembangkan oleh Google.
   Skor kemiripan: 0.3999


...

Kamu berkata: Apakah anda merasa le

Top jawaban paling relevan:

1. Pertanyaan mirip: Apakah Anda merasa leher Anda kaku?
   Jawaban       : Saya mengalami demam tinggi disertai mual.
   Skor kemiripan: 0.6583

2. Pertanyaan mirip: Apakah Anda merasa kesulitan berbicara?
   Jawaban       : Ya, saya merasa kesulitan untuk berbicara, lidah terasa berat.
   Skor kemiripan: 0.6165

3. Pertanyaan mirip: Apakah Anda merasa kulit Anda berubah atau mengalami ruam?
   Jawaban       : Saya mengalami ruam pada kulit dan merasa lelah.
   Skor kemiripan: 0.5773
Silakan bicara...
Mendengarkan...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Kamu berkata: Apakah anda merasa Lemah atau dehidrasi

Top jawaban paling relevan:

1. Pertanyaan mirip: Apakah Anda merasa lemah atau dehidrasi?
   Jawaban       : Saya merasa sangat lemah dan dehidrasi.
   Skor kemiripan: 0.9638

2. Pertanyaan mirip: Apakah Anda merasa mual atau muntah?
   Jawaban       : Ya, saya merasa mual dan terkadang muntah.
   Skor kemiripan: 0.7118

3. Pertanyaan mirip: Apakah Anda sering merasa lelah atau pusing?
   Jawaban       : Sering merasa sangat lelah dan pusing.
   Skor kemiripan: 0.6304
Silakan bicara...
Mendengarkan...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Kamu berkata: selesai


Penjelasan Singkat
------------
>Epoch hanya relevan jika kamu ingin melatih (training) model machine learning/deep learning dari awal (misal: fine-tuning SBERT di dataset sendiri).
Dalam kode kamu, SBERT hanya digunakan untuk inference (mengambil embedding), bukan untuk training ulang. Model SBERT yang dipakai sudah pre-trained dari HuggingFace.
BM25 tidak ada proses training apalagi epoch, hanya membuat index dari data.

1. Tahapan Preprocessing
- Preprocessing biasanya mengacu pada pembersihan, normalisasi, tokenisasi, stemming, dll.<br>
- Pada kode kamu, preprocessing hanya berupa:
    - Tokenisasi sederhana: q.lower().split() untuk BM25 (memecah kalimat jadi kata dan lowercase).
    - Tidak ada stemming, stopword removal, atau normalisasi lanjutan.
- Tidak ada proses cleaning data secara mendalam.

2. Tahapan Training / Epoch
- Epoch adalah istilah dalam pelatihan model (training), di mana seluruh dataset dilalui satu kali untuk mengupdate bobot model.
- Pada kode kamu:
    - BM25: Tidak butuh training/epoch, hanya membuat index berdasarkan token.
    - SBERT: Model SentenceTransformer sudah pre-trained dan hanya digunakan untuk melakukan encoding (inference) saja, bukan training ulang.
    - Tidak ada proses training, fitting, atau epoch yang dilakukan pada dataset kamu.

3. Tahapan yang Ada di Kode
- Load Dataset (read file JSON).
- Preprocessing ringan: Tokenisasi dan lowercase untuk BM25.
- Indexing: Membuat objek BM25 dan menghasilkan embedding SBERT.
- Inference: Mengambil pertanyaan user, melakukan pencarian hybrid, dan mengembalikan jawaban.
- STT & TTS: Hanya untuk input/output, tidak ada preprocessing data.

4. Kesimpulan
- Tidak ada tahapan preprocessing yang kompleks.
- Tidak ada pelatihan model (epoch/training).
- Semua model yang digunakan (BM25 & SBERT) hanya untuk pencarian dan inference, bukan training ulang.

Tidak, tidak diharuskan menggunakan epoch jika konteks penelitian atau aplikasi kamu adalah chatbot retrieval-based seperti di kode yang kamu pakai (BM25 + SBERT untuk pencocokan pertanyaan dan jawaban).

Penjelasan Singkat =
- Epoch hanya relevan jika kamu ingin melatih (training) model machine learning/deep learning dari awal (misal: fine-tuning SBERT di dataset sendiri).
- Dalam kode kamu, SBERT hanya digunakan untuk inference (mengambil embedding), bukan untuk training ulang. Model SBERT yang dipakai sudah pre-trained dari HuggingFace.
- BM25 tidak ada proses training apalagi epoch, hanya membuat index dari data.

Dalam Penelitian/Tesis
- Jika penelitian kamu berfokus pada implementasi, perbandingan retrieval, atau evaluasi sistem chatbot berbasis FAQ (bukan pada pelatihan model baru), maka tidak perlu ada proses epoch.
- Kamu cukup jelaskan bahwa model SBERT yang digunakan sudah pre-trained dan hanya dipakai untuk pencocokan semantic, serta proses retrieval dilakukan dengan hybrid BM25 + SBERT.

Kapan Butuh Epoch?
- Jika kamu ingin melatih model baru (misal: fine-tune SBERT dengan data sendiri, atau buat neural retriever baru), barulah kamu perlu epoch dan tahapan training.
- Biasanya ini untuk penelitian yang ingin membandingkan hasil fine-tuned model vs pre-trained model.

Kesimpulan
- Tidak perlu epoch kalau hanya melakukan retrieval seperti kode di atas.
- Epoch hanya diperlukan untuk training/fine-tuning model, bukan untuk aplikasi retrieval-based FAQ chatbot seperti kode kamu.

Saran untuk penulisan tesis:
- Jelaskan bahwa model SBERT yang digunakan sudah “pre-trained” dan digunakan hanya untuk tahap pencocokan semantic (inference), sehingga tidak ada proses pelatihan (training) dan epoch dalam implementasi ini. Jelaskan juga keuntungan dan keterbatasan pendekatan seperti ini.

-----------------------------------------------
## METODOLOGI PENELITIAN

### 3.1 Desain Penelitian  
Penelitian ini menggunakan desain penelitian rekayasa perangkat lunak untuk membangun sebuah sistem chatbot berbasis hybrid retrieval pada Frequently Asked Questions (FAQ) dengan dukungan teknologi Speech-to-Text (STT) dan Text-to-Speech (TTS). Sistem dirancang untuk menjawab pertanyaan pengguna secara otomatis baik melalui input teks maupun suara, serta memberikan respons dalam bentuk teks dan suara.

### 3.2 Alur Penelitian  
Alur metodologi penelitian yang digunakan meliputi tahapan sebagai berikut:  
1. Studi literatur dan pengumpulan data FAQ.
2. Preprocessing data dan pembuatan dataset.
3. Implementasi chatbot hybrid (BM25 + SBERT).
4. Integrasi fitur STT dan TTS.
5. Evaluasi sistem.

### 3.3 Pengumpulan Data  
Data yang digunakan berupa kumpulan pertanyaan dan jawaban (FAQ) yang diperoleh dari sumber resmi institusi atau website terkait. Data disimpan dalam format JSON untuk memudahkan pemrosesan.

### 3.4 Preprocessing Data  
Preprocessing yang dilakukan pada data FAQ meliputi:  
- Lowercasing (mengubah seluruh teks menjadi huruf kecil)
- Tokenisasi (memecah kalimat menjadi kata) untuk kebutuhan BM25
- Pembuatan embedding menggunakan pre-trained SBERT  
Tidak dilakukan stemming atau stopword removal dalam implementasi dasar ini.

### 3.5 Perancangan Sistem Chatbot Hybrid  
Sistem chatbot terdiri dari dua tahap utama pencarian jawaban:
1. **BM25 Retrieval:**  
   Dilakukan pencarian awal berdasarkan kemiripan kata kunci (lexical matching) untuk mendapatkan kandidat pertanyaan yang relevan.
2. **SBERT Reranking:**  
   Kandidat hasil BM25 kemudian direranking berdasarkan kemiripan semantic menggunakan SBERT (Sentence-BERT) yang sudah pre-trained. Proses ini tanpa pelatihan ulang (training/fine-tuning).

### 3.6 Integrasi STT dan TTS  
- **Speech-to-Text (STT):** Menggunakan library SpeechRecognition untuk mengubah input suara menjadi teks.
- **Text-to-Speech (TTS):** Menggunakan library pyttsx3 untuk memb
